## Sets Won Function

In [4]:
player_name = 'Rudy Quan'

df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
season_matches = df[df['Date'].dt.year == 2025]
player_matches = season_matches[(season_matches['Player1'] == player_name) | 
                                (season_matches['Player2'] == player_name)].copy()

player_matches

,Event Name,Date,Player1,Player1 UTR,Player2,Player2 UTR,Score
2,"Dual Match: University of California, Los Ange...",2025-04-26,Aidan Kim,13.75,Rudy Quan,13.67,"7-6(3), 6-2"
10,Dual Match: Michigan State University vs Unive...,2025-04-25,Rudy Quan,13.67,Aristotelis THANOS,13.61,"4-6, 3-1"
17,Dual Match: University of Michigan vs Universi...,2025-04-24,Rudy Quan,13.67,William Cooksey,12.00,"6-3, 5-6"
23,"Dual Match: University of California, Los Ange...",2025-04-19,Rudy Quan,13.67,Calvin MUELLER,13.00,"3-6, 6-3, 6-1"
25,"Dual Match: University of California, Los Ange...",2025-04-17,Michael Minasyan,12.00,Rudy Quan,13.67,"2-6, 6-3"
32,Dual Match: Michigan State University vs Unive...,2025-04-12,Rudy Quan,13.67,Ozan Baris,13.83,"6-1, 6-2"
36,"Dual Match: University of California, Los Ange...",2025-04-10,Rudy Quan,13.67,William Cooksey,12.00,"6-3, 6-3"
44,"Dual Match: University of California, Los Ange...",2025-04-05,Rudy Quan,13.67,Charl Morgan,12.00,"6-3, 6-3"
48,Dual Match: Ohio State University vs Universit...,2025-04-03,Alexander Bernard,13.33,Rudy Quan,13.67,"7-6(5), 0-6"
52,"Dual Match: University of California, Los Ange...",2025-04-03,Alexander Bernard,13.33,Rudy Quan,13.67,"7-6(5), 0-6, 3-1"


In [3]:
import pandas as pd

df = pd.read_csv("/Users/cjgimena/Desktop/github/consulting-spring2025/data/mens/mens_results.csv")

def get_sets_won(player_name, season_year=2025):
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    season_matches = df[df['Date'].dt.year == season_year]
    player_matches = season_matches[(season_matches['Player1'] == player_name) | 
                                    (season_matches['Player2'] == player_name)].copy()
    sets_won = 0
    for _, row in player_matches.iterrows():
        if pd.isna(row['Score']) or row['Score'].strip() == "":
            continue
        sets = row['Score'].split(", ")

        for set_score in sets:
            if "-" not in set_score:
                continue

            try:
                player1_score = int(set_score.split("-")[0].split("(")[0].strip())
                player2_score = int(set_score.split("-")[1].split("(")[0].strip())
            except ValueError:
                continue

            # Needs to account for unfinished sets (Matches that dont end with 6-*, 7-5, 7-6) - Cj
            if ((player1_score >= 6 or player2_score >= 6) and abs(player1_score - player2_score) >= 2) or ((player1_score == 7 and player2_score == 6) or (player1_score == 6 and player2_score == 7)):
                if row['Player1'] == player_name and player1_score > player2_score:
                    sets_won += 1
                elif row['Player2'] == player_name and player2_score > player1_score:
                    sets_won += 1

    return {"Player": player_name, "Sets Won": sets_won}

print(get_sets_won("Rudy Quan"))

{'Player': 'Rudy Quan', 'Sets Won': 30}


In [5]:
def extract_sets(score):
    if pd.isna(score) or score.strip() == "":
        return []  # Return an empty list if the score is NaN or empty
    return score.split(", ")

def count_sets_won(sets, player_name, player1, player2):
    sets_won = 0
    for set_score in sets:
        if "-" not in set_score:
            continue

        try:
            # Check for 10-point tiebreaker format (e.g., "1-0(3)" or "0-1(7)")
            if "(" in set_score:
                tie_split = set_score.split("(")[0]  # Get the part before "("
                player1_score = int(tie_split.split("-")[0].strip())
                player2_score = int(tie_split.split("-")[1].strip())
                
                # A valid 10-point tiebreaker should have a 1-0 or 0-1 structure
                if (player1_score == 1 and player2_score == 0) or (player1_score == 0 and player2_score == 1):
                    if player1 == player_name and player1_score == 1:
                        sets_won += 1
                    elif player2 == player_name and player2_score == 1:
                        sets_won += 1
                    continue
            
            # Standard set format (e.g., "6-4", "7-6")
            player1_score = int(set_score.split("-")[0].split("(")[0].strip())
            player2_score = int(set_score.split("-")[1].split("(")[0].strip())
        except ValueError:
            continue

        # Check for valid set completion
        if ((player1_score >= 6 or player2_score >= 6) and abs(player1_score - player2_score) >= 2) or \
           ((player1_score == 7 and player2_score == 6) or (player1_score == 6 and player2_score == 7)):
            if player1 == player_name and player1_score > player2_score:
                sets_won += 1
            elif player2 == player_name and player2_score > player1_score:
                sets_won += 1

    return sets_won


In [6]:
def get_sets_won(df, player_name):

    # Apply the function to the 'Score' column
    df['Sets'] = df['Score'].apply(extract_sets)

    # Apply the function to the DataFrame
    df['sets_won'] = df.apply(lambda row: count_sets_won(row['Sets'], player_name, row['Player1'], row['Player2']), axis=1)
    
    return df['sets_won'].sum()


In [7]:
get_sets_won(player_matches, player_name)

31